# File Operations



In [ ]:
!nvidia-smi #ekran kartı bilgilerimi gösteriyor

Tue Aug 24 00:50:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    70W / 149W |   1491MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cp -r /content/drive/MyDrive/InternP2/data.zip . #bu işlemler data.zip'i colab'ın içine kopyalıyor
!unzip -q data.zip #zip'ten çıkarıyor
!rm data.zip #zip'i siliyor


replace data/jsons/cfc_001106.png.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


# Json2Mask-Line

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Jun 26 00:44:19 2021

@author: aycaburcu
"""
import os
import cv2
import tqdm
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from constant import *
from tqdm import tqdm_notebook

jsons=os.listdir(JSON_DIR)#List created with names of json files in ann folder



for json_name in tqdm_notebook(jsons):#access the elements in the json list
    json_path = os.path.join(JSON_DIR, json_name)#Merged json_dir with json_name and created file path
    json_file = open(json_path, 'r')#file reading process
    json_dict=json.load(json_file)#Contents of json file converted to dict data type
    mask=np.zeros((json_dict["size"]["height"],json_dict["size"]["width"]), dtype=np.uint8)
   
    
    
    mask_path = os.path.join(MASK_LINE_DIR, json_name[:-5])
    # The values of the object keys in the dicts that we obtained from each 	json file have been added to the list.
    
    for obj in json_dict["objects"]:# To access each list inside the json_objs list
                         
        if obj['classTitle']=='Solid Line':
           cv2.polylines(mask,np.array([obj['points']['exterior']],dtype=np.int32),False,color=1,thickness=14)
 
        elif obj['classTitle']=='Dashed Line':       
               cv2.polylines(mask,np.array([obj['points']['exterior']],dtype=np.int32),False,color=2,thickness=9)
    
    cv2.imwrite(mask_path,mask.astype(np.uint8))#Print filled masks in mask_path with imwrite

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/8554 [00:00<?, ?it/s]

# Json2Mask-Polygon


In [ ]:
"""
Created on Sat Jun 26 00:44:19 2021
@author: aycaburcu
"""
import os
import cv2
import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt
from constant import *

jsons=os.listdir(JSON_DIR)#List created with names of json files in ann folder



for json_name in tqdm.tqdm(jsons):#access the elements in the json list
    json_path = os.path.join(JSON_DIR, json_name)#Merged json_dir with json_name and created file path
    json_file = open(json_path, 'r')#file reading process
    json_dict=json.load(json_file)#Contents of json file converted to dict data type
    mask=np.zeros((json_dict["size"]["height"],json_dict["size"]["width"]), dtype=np.uint8)
   
    
    mask_path = os.path.join(MASK_POLYGON_DIR, json_name[:-5])
    # The values of the object keys in the dicts that we obtained from each 	json file have been added to the list.
    
    for obj in json_dict["objects"]:# To access each list inside the json_objs list
        
        
        if obj['classTitle']=='Freespace':#Objects whose classtitle is freespace
            
            cv2.fillPoly(mask,np.array([obj['points']['exterior']],dtype=np.int32),color=1)
          
            if obj['points']['interior'] !=[]: #Checking if interior list is not empty
                for interior in obj['points']['interior']:
                         #Converting to int32 because there are floats between points
                         cv2.fillPoly(mask,np.array([interior],dtype=np.int32),color=0)
                         

    
    cv2.imwrite(mask_path,mask.astype(np.uint8))#Print filled masks in mask_path with imwrite

100%|██████████| 8554/8554 [01:04<00:00, 133.38it/s]


# Train

In [ ]:
#from line_Unet import LINE_NET
#from polygon_model import POL_NET
from line_SegNet import SegNet
from preprocessing import tensorize_image, tensorize_mask, image_mask_check
import os
import glob
import numpy as np
import torch.nn as nn
import torch.optim as optim
from constant import *
import tqdm
import torch
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from tqdm import tqdm_notebook

def train(valid_size,test_size,batch_size,epochs,cuda,input_shape,n_classes,mask_dir,model,model_save,train_if):
    global test_input_path_list
    ######### DIRECTORIES #########
    SRC_DIR = os.getcwd()
    
    ###############################
    
    
    # PREPARE IMAGE AND MASK LISTS
    image_path_list = glob.glob(os.path.join(IMG_DIR, '*'))
    image_path_list.sort()
    
    mask_path_list = glob.glob(os.path.join(MASK_DIR, '*'))
    mask_path_list.sort()
    
    # DATA CHECK
    image_mask_check(image_path_list, mask_path_list)
    #Checked whether the elements in mask_path_list and image_path_list list are the same.
    
    
    
    # SHUFFLE INDICES
    indices = np.random.permutation(len(image_path_list))
    #A random array of permutations for the length of the image_path_list steps_per_epoch = len (train_input_path_list) // batch_size is created
    
    
    # DEFINE TEST AND VALID INDICES
    test_ind  = int(len(indices) * test_size)#Multiply indices length by test_size and assign it to an int-shaped variable
    valid_ind = int(test_ind + len(indices) * valid_size)
    
    # SLICE TEST DATASET FROM THE WHOLE DATASET
    test_input_path_list = image_path_list[:test_ind] #Get 0 to 476 elements of the image_path_list list
    test_label_path_list = mask_path_list[:test_ind]#Get 0 to 476 elements of the mask_path_list list
    
    # SLICE VALID DATASET FROM THE WHOLE DATASET
    valid_input_path_list = image_path_list[test_ind:valid_ind]#Get 476 to 1905 elements of the image_path_list list
    valid_label_path_list = mask_path_list[test_ind:valid_ind]#Get 476 to 1905 elements of the mask_path_list list
    
    # SLICE TRAIN DATASET FROM THE WHOLE DATASET
    train_input_path_list = image_path_list[valid_ind:]#Get the elements of the image_path_list list from 1905 to the last element
    train_label_path_list = mask_path_list[valid_ind:]#Get the elements of the mask_path_list list from 1905 to the last element
    #burada yukarıda vermiş olduğumuz test verisi için tüm datanın 0.1 ve validation verisi tüm datanın 0.3 içermeli
    #Here, for the test data we have given above, all the data should contain 0.1 and all the validation data should contain 0.3, 
    #but both of them do not belong to the same data data.
    
    # train_input_path_list.extend(aug_path_list)
    # train_label_path_list.extend(aug_mask_path_list)
    
    
    
    
    
    if train_if:    
        steps_per_epoch = len(train_input_path_list)//batch_size
        # Find how many times to do it by dividing the length of the train data (training data) by batch_size
        #in an epoch, a data string in the dataset goes to the end in neural networks
        #It then waits there until the batch reaches you, the error rate is calculated after the data reaches the end
        #Divide the training data set by 4 since our batch_size is 4
        
        # CALL MODEL
        model = model
        #Enter parameters into model and assign output to variable
        
        # DEFINE LOSS FUNCTION AND OPTIMIZER
        
    
        criterion = nn.CrossEntropyLoss()#Creates a criterion that measures the Binary Cross Entropy between target and output:
        #BCELoss is an acronym for Binary CrossEntropyLoss, a special case of BCOMoss CrossEntropyLoss used only for two categories of problems.
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        #Commonly used momentum beta coefficient is 0.9.
        #lr=learning rate
        
        # IF CUDA IS USED, IMPORT THE MODEL INTO CUDA
        if cuda:
            model = model.cuda()
        
        val_losses=[]
        train_losses=[]
        # TRAINING THE NEURAL NETWORK
        for epoch in tqdm_notebook(range(epochs)):
        
            running_loss = 0
            #In each epoch, images and masks are mixed randomly in order not to output images sequentially.
            pair_IM=list(zip(train_input_path_list,train_label_path_list))
            np.random.shuffle(pair_IM)
            unzipped_object=zip(*pair_IM)
            zipped_list=list(unzipped_object)
            train_input_path_list=list(zipped_list[0])
            train_label_path_list=list(zipped_list[1])
            
            for ind in tqdm_notebook(range(steps_per_epoch)):
                batch_input_path_list = train_input_path_list[batch_size*ind:batch_size*(ind+1)]
                #train_input_path_list [0: 4] gets first 4 elements on first entry
                #in the second loop train_input_list [4: 8] gets the second 4 elements
                #element advances each time until batch_size
                batch_label_path_list = train_label_path_list[batch_size*ind:batch_size*(ind+1)]
                batch_input = tensorize_image(batch_input_path_list, input_shape, cuda)
                batch_label = tensorize_mask(batch_label_path_list, input_shape, n_classes, cuda)
                #Our data that we will insert into the model in the preprocess section is prepared by entering the parameters.
                
                optimizer.zero_grad()#gresets the radian otherwise accumulation occurs on each iteration
                # Manually reset gradients after updating Weights
                
                
                outputs = model(batch_input) # Give the model batch_input as a parameter and assign the resulting output to the variable.
                
    
                # Forward passes the input data
                batch_label= torch.argmax(batch_label, dim=1)
                loss = criterion(outputs, batch_label)
                loss.backward()# Calculates the gradient, how much each parameter needs to be updated
                optimizer.step()# Updates each parameter according to the gradient
        
                running_loss += loss.item()# loss.item () takes the scalar value held in loss.
        
               
                #validation 
                if ind == steps_per_epoch-1:
                    
                    train_losses.append(running_loss)
                    print('training loss on epoch {}: {}'.format(epoch, running_loss))
                    val_loss = 0
                    for (valid_input_path, valid_label_path) in zip(valid_input_path_list, valid_label_path_list):
                        batch_input = tensorize_image([valid_input_path], input_shape, cuda)
                        batch_label = tensorize_mask([valid_label_path], input_shape, n_classes, cuda)
                        batch_label= torch.argmax(batch_label, dim=1)
                        outputs = model(batch_input)
                        loss = criterion(outputs, batch_label)
                        val_loss += loss.item()
                        val_losses.append(val_loss)
                        break
        
                    print('validation loss on epoch {}: {}'.format(epoch, val_loss))
                    
        torch.save(model,model_save)
        print("Model Saved!")
        
        
        def draw_graph(val_losses,train_losses,epochs):
            norm_validation = [float(i)/sum(val_losses) for i in val_losses]
            norm_train = [float(i)/sum(train_losses) for i in train_losses]
            epoch_numbers=list(range(1,epochs+1,1))
            plt.figure(figsize=(12,6))
            plt.subplot(2, 2, 1)
            plt.plot(epoch_numbers,norm_validation,color="red") 
            plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
            plt.title('Train losses')
            plt.subplot(2, 2, 2)
            plt.plot(epoch_numbers,norm_train,color="blue")
            plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
            plt.title('Validation losses')
            plt.subplot(2, 1, 2)
            plt.plot(epoch_numbers,norm_validation, 'r-',color="red")
            plt.plot(epoch_numbers,norm_train, 'r-',color="blue")
            plt.legend(['w=1','w=2'])
            plt.title('Train and Validation Losses')
            plt.gca().xaxis.set_major_locator(mticker.MultipleLocator(1))
            
            
            plt.show()
        
        draw_graph(val_losses,train_losses,epochs)
    
#########POLYGON###############
######### PARAMETERS ##########
valid_size = 0.3
test_size  = 0.1
batch_size = 4
epochs = 20
cuda = True
input_shape = (224, 224)
n_classes = 4
MASK_DIR=MASK_POLYGON_DIR
model=FoInternNet(input_shape,n_classes)
model_save='models/best_polygon_model.pt'
train_if=True
# ############################### 
# #polygon_model
train(valid_size,test_size,batch_size,epochs,cuda,input_shape,n_classes,MASK_DIR,model,model_save)



#########LINE###############
######### PARAMETERS ##########
valid_size = 0.3
test_size  = 0.1
batch_size = 4
epochs = 30
cuda = True
input_shape = (224, 224)
n_classes = 3
MASK_DIR=MASK_LINE_DIR
model=SegNet(n_classes)
model_save='models/best_line_model.pt'
train_if=True
############################### 
#polygon_model
train(valid_size,test_size,batch_size,epochs,cuda,input_shape,n_classes,MASK_DIR,model,model_save,train_if)


ModuleNotFoundError: ignored

In [ ]:
!zip -r best_line_model.zip models
!cp best_line_model.zip /content/drive/MyDrive/InternP2

#Predict

In [ ]:
import torch.nn as nn
import torch.optim as optim
from constant import *
import tqdm
import torch
from preprocessing import tensorize_image, tensorize_mask, image_mask_check
import cv2
from train import *
from PIL import Image



model_freespace= torch.load('/content/drive/MyDrive/models/Unet_1.pt')
model_line=torch.load('/content/drive/MyDrive/models/best_line_model.pt',map_location='cuda:0')
model_freespace=model_freespace.eval()
model_line=model_line.eval()
input_shape=(224,224)
cuda=True
if cuda:
    model_line = model_line.cuda()
    model_freespace=model_freespace.cuda()


for i in tqdm.tqdm(range(len(test_input_path_list))):
    batch_test = test_input_path_list[i:i+1]
    test_input_line = tensorize_image(batch_test, input_shape, cuda)
    test_input_freespace=tensorize_image(batch_test, input_shape, cuda)
    outs_freespace = model_freespace(test_input_freespace)
    outs_line = model_line(test_input_line)
    out_freespace=torch.argmax(outs_freespace,axis=1)
    out_line=torch.argmax(outs_line,axis=1)
    out_freespace_cpu = out_freespace.cpu()
    out_line_cpu=out_line.cpu()
    outputs_list_freespace=out_freespace_cpu.detach().numpy()
    outputs_list_line=out_line_cpu.detach().numpy()
    mask_freespace=np.squeeze(outputs_list_freespace,axis=0)
    mask_line=np.squeeze(outputs_list_line,axis=0)
    mask_uint8_line = mask_line.astype('uint8')
    mask_uint8_freespace = mask_freespace.astype('uint8')
    mask_line= cv2.resize(mask_uint8_line, (1920, 1208),interpolation=cv2.INTER_NEAREST)
    mask_freespace= cv2.resize(mask_uint8_freespace, (1920, 1208),interpolation=cv2.cv2.INTER_CUBIC)
        
    img=cv2.imread(batch_test[0])
    img=cv2.resize(img,(1920, 1208))
    mask_ind   = mask_line == 1
    mask_ind   = mask_freespace == 1
    cpy_img  = img.copy()
    
    img[mask_freespace==1,:] = (255, 0, 125)


    img[mask_line==1,:]=(0, 0, 255)
    img[mask_line==2,:]=(38, 255, 255)
    
    
    opac_image=(img/2+cpy_img/2).astype(np.uint8)
    predict_name=batch_test[0]
    predict_path=predict_name.replace('images', 'predict')
    cv2.imwrite(predict_path,opac_image.astype(np.uint8))



100%|██████████| 427/427 [03:16<00:00,  2.17it/s]


In [ ]:
!zip -r predict_full.zip data/predict
!cp predict_full.zip /content/drive/MyDrive/InternP2

updating: data/predict/ (stored 0%)
updating: data/predict/cfc_000321.jpg (deflated 1%)
updating: data/predict/cfc_000490.jpg (deflated 1%)
updating: data/predict/cfc_000450.jpg (deflated 1%)
updating: data/predict/cfc_000482.jpg (deflated 1%)
updating: data/predict/cfc_000475.jpg (deflated 1%)
updating: data/predict/cfc_000320.jpg (deflated 1%)
updating: data/predict/cfc_000509.jpg (deflated 1%)
updating: data/predict/cfc_000418.jpg (deflated 1%)
updating: data/predict/cfc_000310.jpg (deflated 1%)
updating: data/predict/cfc_000340.jpg (deflated 1%)
updating: data/predict/cfc_000265.jpg (deflated 1%)
updating: data/predict/cfc_000612.jpg (deflated 1%)
updating: data/predict/cfc_000647.jpg (deflated 1%)
updating: data/predict/cfc_000264.jpg (deflated 1%)
updating: data/predict/cfc_000592.jpg (deflated 1%)
updating: data/predict/cfc_000648.jpg (deflated 1%)
updating: data/predict/cfc_000572.jpg (deflated 1%)
updating: data/predict/cfc_000316.jpg (deflated 1%)
updating: data/predict/cfc_0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
